<a href="https://colab.research.google.com/github/agusnieto77/taller_pict_2019/blob/main/Colabs/ACEP_funciones_desarrollo_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 3

In [ ]:
devtools::install_github("agusnieto77/ACEP")

In [ ]:
require(ACEP)

In [ ]:
acep_context("La UTA inició un paro de colectivos por tiempo indeterminado", "paro")

In [ ]:
acep_context(acep_clean("La UTA inició un paro de colectivos por tiempo indeterminado"), "paro")

In [ ]:
acep_context(acep_clean("La UTA inició un paro de colectivos por tiempo indeterminado"), "paro", izq = 2, der = 2)

In [ ]:
acep_context(acep_clean("La UTA inició un paro de colectivos por tiempo indeterminado"), "paro", izq = 2, der = 3)

In [ ]:
acep_context(acep_clean("La UTA inició un paro de colectivos por tiempo indeterminado"), "inicio paro colectivos", izq = 1, der = 3)

In [9]:
texto <-     
  c("La UTA inició un paro de colectivos por tiempo indeterminado.", 
    "El gremio que nuclea a los choferes lo resolvió por un atraso salarial.",
    "La medida de fuerza comenzó a partir de las 18.",
    "Por un atraso salarial, la Unión Tranviarios Automotor (UTA) inició 
    un paro por tiempo indeterminado a partir de las 18 de este lunes, 
    según confirmaron fuentes del gremio que nuclea a los choferes.")

In [ ]:
acep_context(acep_clean(texto), "paro", izq = 2, der = 2)

In [11]:
acep_context(acep_clean(texto), c("paro", "medida fuerza"), izq = 2, der = 2)

doc_id,contexto
<int>,<chr>
1,uta inicio | paro | colectivos tiempo
4,uta inicio | paro | tiempo indeterminado
3,| medida fuerza comenzo partir | comenzo partir


In [12]:
texto2 <-     
  c("La UTA inició un paro por tiempo indeterminado. 
    El paro de la UTA no tiene relación con el paro de Moyano.", 
    "El gremio que nuclea a los choferes resolvió el paro por un atraso salarial.",
    "La medida de fuerza comenzó a partir de las 18.",
    "Por un atraso salarial, la Unión Tranviarios Automotor (UTA) inició 
    un paro por tiempo indeterminado a partir de las 18 de este lunes, 
    según confirmaron fuentes del gremio que nuclea a los choferes.")

In [13]:
acep_context(acep_clean(texto2), c("paro", "medida fuerza"), izq = 4, der = 4)

doc_id,contexto
<int>,<chr>
1,uta inicio | paro | tiempo indeterminado | paro | uta
1,relacion | paro | moyano
2,gremio nuclea choferes resolvio | paro | atraso salarial
4,tranviarios automotor uta inicio | paro | tiempo indeterminado partir segun
3,| medida fuerza comenzo partir | comenzo partir


### Posible solución

In [14]:
acep_context <- function(texto, clave, izq = 1, der = 1){
  nwi <- "[[:graph:]]*[[:space:]]*"
  nwd <- "[[:space:]]*[[:graph:]]*"
  lista_frases <- c()
  doc_id <- c()
  for (c in clave) {
    claveb <- paste0(c, "[[:graph:]]*", collapse = "|")
    capturar <- paste0("(", paste0(rep(nwi,izq), collapse = ""),
                       claveb, paste0(rep(nwd, der), collapse = ""), ")")
    for (i in seq_along(texto)) {
      vector <- unlist(regmatches(texto[i], gregexpr(capturar, texto[i])))
      doc_id <- append(doc_id, rep(i, length(vector)))
      claves <- unlist(regmatches(texto[i], gregexpr(claveb, texto[i])))
      lista <- sapply(seq_along(vector), function(x) gsub(claveb, paste0("\\| ", claves[x], " \\|"), vector[x]))
      lista_frases <- unlist(append(lista_frases, lista))
    }
  }
  lista_frases <- data.frame(doc_id = doc_id, contexto = lista_frases)
  return(lista_frases)
}

In [15]:
acep_context(acep_clean(texto2), c("paro", "medida fuerza"), izq = 4, der = 4)

doc_id,contexto
<int>,<chr>
1,uta inicio | paro | tiempo indeterminado | paro | uta
1,relacion | paro | moyano
2,gremio nuclea choferes resolvio | paro | atraso salarial
4,tranviarios automotor uta inicio | paro | tiempo indeterminado partir segun
3,| medida fuerza | comenzo partir


In [16]:
acep_context <- function(texto, clave, izq = 1, der = 1){
  nwi <- "[[:graph:]]*[[:space:]]*"
  nwd <- "[[:space:]]*[[:graph:]]*"
  lista_frases <- c()
  doc_id <- c()
  oraciones_id <- c()
  
  for (c in clave) {
    claveb <- paste0(c, "[[:graph:]]*", collapse = "|")
    capturar <- paste0("(", paste0(rep(nwi,izq), collapse = ""),
                       claveb, paste0(rep(nwd, der), collapse = ""), ")")
    
    for (o in seq_along(texto)) {
      oraciones <- unlist(strsplit(texto[o], "(?<=[a-z]\\.|\\?|\\!)\\s*(?=[A-Z]|\n[A-Z])", perl=T))
      
      for (i in seq_along(oraciones)) {
        vector <- unlist(regmatches(oraciones[i], gregexpr(capturar, oraciones[i])))
        
        for (v in seq_along(vector)) {
          doc_id <- append(doc_id, o)
          oraciones_id <- append(oraciones_id, i)
          claves <- unlist(regmatches(vector[v], gregexpr(claveb, vector[v])))
          lista <- sapply(seq_along(vector[v]), function(x) sub(claveb, paste0("\\| ", claves[x], " \\|"), vector[v][x]))
          lista_frases <- unlist(append(lista_frases, lista))
        }
      }
    }
  }
    
  lista_frases <- data.frame(doc_id = doc_id, oraciones_id = oraciones_id, contexto = lista_frases)
  return(lista_frases)
}

In [17]:
acep_context(acep_clean(texto2), c("paro", "medida fuerza"), izq = 4, der = 4)

doc_id,oraciones_id,contexto
<int>,<int>,<chr>
1,1,uta inicio | paro | tiempo indeterminado paro uta
1,1,relacion | paro | moyano
2,1,gremio nuclea choferes resolvio | paro | atraso salarial
4,1,tranviarios automotor uta inicio | paro | tiempo indeterminado partir segun
3,1,| medida fuerza | comenzo partir


In [18]:
acep_context(texto2, c("paro", "medida de fuerza"), izq = 4, der = 4)

doc_id,oraciones_id,contexto
<int>,<int>,<chr>
1,1,La UTA inició un | paro | por tiempo indeterminado.
1,2,El | paro | de la UTA no
1,2,tiene relación con el | paro | de Moyano.
2,1,los choferes resolvió el | paro | por un atraso salarial.
4,1,Automotor (UTA) inició un | paro | por tiempo indeterminado a
3,1,La | medida de fuerza | comenzó a partir de
